## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

## Config

In [2]:
num_workers = 4
batch_size = 128
val_size = 0.1
n_epochs = 20
print_every = 500

device = torch.device('cuda:0')

### Downloading and Creating Train and Test Sets

In [3]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(root = 'data', 
							              train=True, 
							              download=True,
                            transform=transform)

test_data = datasets.MNIST(root='data', 
						               train=False, 
						               download=True,
                           transform=transform)

## Creating Data Loaders

In [4]:
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(val_size * num_train))

train_index, val_index = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_index)
val_sampler = SubsetRandomSampler(val_index)


train_loader = torch.utils.data.DataLoader(train_data, 
										                       batch_size=batch_size,
                                           sampler = train_sampler, 
                                           num_workers = num_workers)

val_loader = torch.utils.data.DataLoader(train_data, 
										                     batch_size = batch_size,
                                         sampler=val_sampler,
                                         num_workers = num_workers)

test_loader = torch.utils.data.DataLoader(test_data, 
										                      batch_size=batch_size,
                                          num_workers=num_workers)

## Defining the Network Class

In [5]:
class Net(nn.Module):
    def __init__(self, hidden_1=512, hidden_2=512):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 10)
        self.droput = nn.Dropout(0.2)  # dropout prevents overfitting on the training data
        
    def forward(self,x):
        x = x.view(-1, 28*28)
        
        x = F.relu(self.fc1(x))
        x = self.droput(x)
        
        x = F.relu(self.fc2(x))
        x = self.droput(x)
        
        x = self.fc3(x)
        return x

## Instantiating the Network, Optimizer, and Loss Criterion

In [8]:
model = Net(512, 512)

# NOTE: TRNASFER THE MODEL TO GPU BEFORE INSTANTIATING THE OPTIMIZER!!
# model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

## Training Loop

In [11]:
val_loss_min = np.Inf

for epoch in range(n_epochs):
  train_loss = 0
     
  # preparing the model for training
  model.train()
    
  for iteration, (data, label) in enumerate(train_loader):
    optimizer.zero_grad()
#     output = model(data.to(device))
#     loss = criterion(output, label.to(device))
    output = model(data)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()

    # update running train loss 
    train_loss += loss.item() * data.shape[0]
    
    if iteration % print_every == 0:
      is_training = model.training
      val_loss = 0
      model.eval()  # preparing model for evaluation
      
      for data, label in val_loader:
#         output = model(data.to(device))
        output = model(data)
#         loss = criterion(output, label.to(device))
        loss = criterion(output, label)
        # update running validation loss 
        val_loss += loss.item() * data.shape[0]

      val_loss = val_loss / len(val_loader.sampler)

    	# save model if validation loss has decreased
      if val_loss <= val_loss_min:
        print('\tValidation loss decreased ({:.6f} --> {:.6f}). Saving model ...'.format(val_loss_min, val_loss))
        torch.save(model.state_dict(), 'model.pt')
        val_loss_min = val_loss

      model.train(mode=is_training)

  # calculate average trainnig loss over an epoch
  train_loss = train_loss / len(train_loader.sampler)
    
  print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

	Validation loss decreased (inf --> 2.303507). Saving model ...
Epoch: 1 	Training Loss: 2.152233
	Validation loss decreased (2.303507 --> 1.828278). Saving model ...
Epoch: 2 	Training Loss: 1.252378
	Validation loss decreased (1.828278 --> 0.783717). Saving model ...
Epoch: 3 	Training Loss: 0.676357
	Validation loss decreased (0.783717 --> 0.529105). Saving model ...
Epoch: 4 	Training Loss: 0.524082
	Validation loss decreased (0.529105 --> 0.436774). Saving model ...
Epoch: 5 	Training Loss: 0.455806
	Validation loss decreased (0.436774 --> 0.388052). Saving model ...
Epoch: 6 	Training Loss: 0.413485
	Validation loss decreased (0.388052 --> 0.357727). Saving model ...
Epoch: 7 	Training Loss: 0.382494
	Validation loss decreased (0.357727 --> 0.336792). Saving model ...
Epoch: 8 	Training Loss: 0.360652
	Validation loss decreased (0.336792 --> 0.317409). Saving model ...
Epoch: 9 	Training Loss: 0.344794
	Validation loss decreased (0.317409 --> 0.304129). Saving model ...
Epoch: 10

In [12]:
model = Net()
model.load_state_dict(torch.load('model.pt'))
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (droput): Dropout(p=0.2)
)


In [14]:
test_loss = 0.0
num_correct = 0

model.eval()
for data, target in test_loader:
    output = model(data)
    
    pred = torch.argmax(output, dim=1)
    num_correct += torch.sum(pred == target).item()
    
test_acc = num_correct / len(test_loader.sampler)
print('Test Accuracy: {:.2f}%\n'.format(test_acc * 100))

Test Accuracy: 94.04%

